In [18]:
import zipfile
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [19]:
zf = zipfile.ZipFile("playground-series-s4e1.zip")
zf.namelist()

['sample_submission.csv', 'test.csv', 'train.csv']

In [20]:
df_train = pd.read_csv(zf.open("train.csv"))
df_test = pd.read_csv(zf.open("test.csv"))

In [21]:
#Checking to see if any rows/columns are missing any data
df_train.describe()

,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,165034.0000,1.650340e+05,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000
mean,82516.5000,1.569201e+07,656.454373,38.125888,5.020353,55478.086689,1.554455,0.753954,0.497770,112574.822734,0.211599
std,47641.3565,7.139782e+04,80.103340,8.867205,2.806159,62817.663278,0.547154,0.430707,0.499997,50292.865585,0.408443
min,0.0000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,41258.2500,1.563314e+07,597.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74637.570000,0.000000
50%,82516.5000,1.569017e+07,659.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,117948.000000,0.000000
75%,123774.7500,1.575682e+07,710.000000,42.000000,7.000000,119939.517500,2.000000,1.000000,1.000000,155152.467500,0.000000
max,165033.0000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


In [22]:
#Checking if any columns have words that need to be converted to integers/floats for prediction calculations

df_train.drop('Surname', axis =1, inplace=True)
df_train.head(5)

,id,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [23]:
df_dummy_train = pd.get_dummies(df_train, drop_first= False, dtype=float)

df_dummy_train.head(5)

,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0,15674932,668,33.0,3,0.00,2,1.0,0.0,181449.97,0,1.0,0.0,0.0,0.0,1.0
1,1,15749177,627,33.0,1,0.00,2,1.0,1.0,49503.50,0,1.0,0.0,0.0,0.0,1.0
2,2,15694510,678,40.0,10,0.00,2,1.0,0.0,184866.69,0,1.0,0.0,0.0,0.0,1.0
3,3,15741417,581,34.0,2,148882.54,1,1.0,1.0,84560.88,0,1.0,0.0,0.0,0.0,1.0
4,4,15766172,716,33.0,5,0.00,2,1.0,1.0,15068.83,0,0.0,0.0,1.0,0.0,1.0


In [24]:
y = df_dummy_train['Exited']
x = df_dummy_train.drop('Exited', 'CustomerId', axis =1)

TypeError: DataFrame.drop() takes from 1 to 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state=42)

In [ ]:
model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')
model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

print('\nClassification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.87

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.95      0.92     26052
           1       0.75      0.57      0.65      6955

    accuracy                           0.87     33007
   macro avg       0.82      0.76      0.78     33007
weighted avg       0.86      0.87      0.86     33007



In [ ]:
df_dummy_test = pd.get_dummies(df_test, drop_first= False, dtype=float)
df_dummy_test

,id,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,...,Surname_Zubarev,Surname_Zubareva,Surname_Zuev,Surname_Zuyev,Surname_Zuyeva,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,165034,15773898,586,23.0,2,0.00,2,0.0,1.0,160976.75,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,165035,15782418,683,46.0,2,0.00,1,1.0,0.0,72549.27,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,165036,15807120,656,34.0,7,0.00,2,1.0,0.0,138882.09,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,165037,15808905,681,36.0,8,0.00,1,1.0,0.0,113931.57,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,165038,15607314,752,38.0,10,121263.62,1,1.0,0.0,139431.00,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110018,275052,15662091,570,29.0,7,116099.82,1,1.0,1.0,148087.62,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
110019,275053,15774133,575,36.0,4,178032.53,1,1.0,1.0,42181.68,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
110020,275054,15728456,712,31.0,2,0.00,2,1.0,0.0,16287.38,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
110021,275055,15687541,709,32.0,3,0.00,1,1.0,1.0,158816.58,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [25]:
y_test_pred = model.predict(df_dummy_test)
y_test_prob = model.predict_proba(df_dummy_test)[:, 1]

# Create a DataFrame with the predicted labels and probabilities
results_df = pd.DataFrame({'True Labels': y_test, 'Predicted Labels': y_test_pred, 'Predicted Probabilities': y_test_prob})

ValueError: feature_names mismatch: ['id', 'CustomerId', 'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Surname_Abazu', 'Surname_Abbie', 'Surname_Abbott', 'Surname_Abdullah', 'Surname_Abdulov', 'Surname_Abel', 'Surname_Abernathy', 'Surname_Abramov', 'Surname_Abramova', 'Surname_Abramovich', 'Surname_Abramowitz', 'Surname_Abrego', 'Surname_Abron', 'Surname_Achebe', 'Surname_Adams', 'Surname_Adamson', 'Surname_Afamefula', 'Surname_Afamefuna', 'Surname_Afanasyev', 'Surname_Afanasyeva', 'Surname_Agafonova', 'Surname_Aguirre', 'Surname_Ah Mouy', 'Surname_Ahern', 'Surname_Ahmed', 'Surname_Aiken', 'Surname_Aikenhead', 'Surname_Ainsworth', 'Surname_Aitken', 'Surname_Ajuluchukwu', 'Surname_Akabueze', 'Surname_Akeroyd', 'Surname_Akhtar', 'Surname_Akobundu', 'Surname_Aksakova', 'Surname_Aksenov', 'Surname_Aksenova', 'Surname_Aksyonov', 'Surname_Aksyonova', 'Surname_Akubundu', 'Surname_Akudinobi', 'Surname_Alaniz', 'Surname_Alderete',